# TBH Secure Agents - Guardrails Example

This notebook demonstrates how to use the `guardrails` parameter with the TBH Secure Agents framework.

## Installation

First, let's install the latest version of the package:

In [1]:
!pip install --upgrade tbh-secure-agents

## Import the necessary modules

In [2]:
from tbh_secure_agents import Expert, Operation, Squad
import os

/Users/saish/Downloads/tbh_secure_agents/build_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set up your API key

You can either set it as an environment variable or directly in the code:

In [ ]:
# Option 1: Set as environment variable
# os.environ["GOOGLE_API_KEY"] = "your-api-key-here"

# Option 2: Set directly in code
api_key = ""  # Replace with your actual API key

## Create Experts with Template Variables

Notice how we use curly braces `{}` to define template variables that will be replaced by the guardrails values.

In [4]:
researcher = Expert(
    specialty="Research Expert specializing in {topic_area}",
    objective="Research and analyze information about {specific_topic}",
    background="You have extensive knowledge in {topic_area} research.",
    api_key=api_key
)

writer = Expert(
    specialty="Content Writer",
    objective="Create engaging {content_type} based on research findings",
    background="You excel at creating {tone} content for {audience_level} audiences.",
    api_key=api_key
)

2025-05-03 12:47:45,390 - tbh_secure_agents.agent - INFO - Gemini API configured.
2025-05-03 12:47:45,392 - tbh_secure_agents.agent - INFO - Expert 'Research Expert specializing in {topic_area}' initialized with Gemini model 'gemini-2.0-flash-lite' and security profile 'default'.
2025-05-03 12:47:45,393 - tbh_secure_agents.agent - INFO - Gemini API configured.
2025-05-03 12:47:45,394 - tbh_secure_agents.agent - INFO - Expert 'Content Writer' initialized with Gemini model 'gemini-2.0-flash-lite' and security profile 'default'.


## Create Operations with Template Variables and Select Syntax

We can use both simple template variables and the more advanced select syntax:

In [5]:
research_operation = Operation(
    instructions="""
    Research the topic of {specific_topic} within the field of {topic_area}.
    Focus on recent developments and key concepts.
    
    {depth, select,
      basic:Provide a high-level overview suitable for beginners.|  
      intermediate:Include more detailed information for those with some knowledge.|  
      advanced:Provide in-depth analysis for experts in the field.
    }
    """,
    output_format="A comprehensive research summary with key findings",
    expert=researcher
)

writing_operation = Operation(
    instructions="""
    Based on the research findings, create a {content_type} about {specific_topic}.
    
    {tone, select,
      formal:Use a professional, academic tone.|  
      conversational:Use a friendly, approachable tone.|  
      technical:Use precise technical language.
    }
    
    The content should be suitable for a {audience_level} audience.
    """,
    output_format="A well-structured {content_type} with clear sections",
    expert=writer
)

## Create a Squad with the Experts and Operations

In [6]:
research_squad = Squad(
    experts=[researcher, writer],
    operations=[research_operation, writing_operation],
    process="sequential"  # Operations run in sequence
)

print(f"Squad created with {len(research_squad.experts)} experts and {len(research_squad.operations)} operations.")

2025-05-03 12:48:01,980 - tbh_secure_agents.crew - INFO - Squad initialized with 2 experts and 2 operations. Process: sequential, Security Level: standard
Squad created with 2 experts and 2 operations.


## Define Guardrail Inputs

These values will be used to replace the template variables in the experts' attributes and operations' instructions.

In [7]:
guardrail_inputs = {
    "topic_area": "artificial intelligence",
    "specific_topic": "large language models",
    "depth": "intermediate",
    "content_type": "blog post",
    "tone": "conversational",
    "audience_level": "general"
}

## Deploy the Squad with Guardrails

Now we can deploy the squad with the guardrails parameter:

In [8]:
print("Starting squad deployment with guardrails...")
result = research_squad.deploy(guardrails=guardrail_inputs)

print("\nRESULT:")
print("=" * 80)
print(result)
print("=" * 80)

Starting squad deployment with guardrails...
2025-05-03 12:48:08,384 - tbh_secure_agents.crew - INFO - Squad deployment initiated...
2025-05-03 12:48:08,388 - tbh_secure_agents.crew - INFO - Squad deployment with guardrails: ['topic_area', 'specific_topic', 'depth', 'content_type', 'tone', 'audience_level']
2025-05-03 12:48:08,392 - tbh_secure_agents.crew - INFO - Operation '
    Research the topic of {sp...' already assigned to Expert 'Research Expert specializing in {topic_area}'
2025-05-03 12:48:08,393 - tbh_secure_agents.crew - INFO - Executing operation 1/2: '
    Research the topic of {sp...'
2025-05-03 12:48:08,396 - tbh_secure_agents.task - INFO - Operation '
    Research the topic of {specific_topic} within...' starting execution by expert 'Research Expert specializing in {topic_area}'.
2025-05-03 12:48:08,396 - tbh_secure_agents.agent - INFO - Expert 'Research Expert specializing in {topic_area}' starting operation execution: 
    Research the topic of {specific_topic} within

## Experiment with Different Guardrail Values

Try changing the values in the guardrail_inputs dictionary to see how it affects the output:

In [ ]:
# Example: Change to a different topic and depth
new_guardrail_inputs = {
    "topic_area": "cybersecurity",
    "specific_topic": "zero trust architecture",
    "depth": "basic",
    "content_type": "technical report",
    "tone": "formal",
    "audience_level": "professional"
}

# Uncomment to run with new guardrails
# result = research_squad.deploy(guardrails=new_guardrail_inputs)
# print("\nNEW RESULT:")
# print("=" * 80)
# print(result)
# print("=" * 80)